In [ ]:
import json
import os
import time

import matplotlib as mpl
import matplotlib.colors
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
from sklearn.utils.extmath import randomized_svd

DATA_DIR = 'data/' # Intermediates
IMAGE_DIR = DATA_DIR + 'apes/'
OUTPUT_DIR = 'output/' # Ready to use / upload

os.makedirs(f'{OUTPUT_DIR}blog', exist_ok=True)
os.makedirs(f'{OUTPUT_DIR}nft/opensea', exist_ok=True)
os.makedirs(f'{OUTPUT_DIR}nft/eigenapes', exist_ok=True)

mpl.rc('image', cmap='gray')
mpl.rc('figure', dpi=200, figsize=[8, 4])
mpl.rc('savefig', facecolor='w')

IMSHOW_PARAMS = {
    'norm': matplotlib.colors.Normalize(0, 255)
}

In [ ]:
print('Loading images...')

filenames = os.listdir(IMAGE_DIR)
count = len(filenames)

im = Image.open(f'{IMAGE_DIR}{filenames[0]}')

w, h = im.size
X = np.ndarray((h * w, count), dtype=np.float32)

for i in range(count):
    im = Image.open(f'{IMAGE_DIR}{filenames[i]}').convert('L')
    X[:, i] = im.getdata()

print("Subtracting mean...")

X_mean = np.mean(X, 1)

for i in range(count):
    X[:, i] -= X_mean

print('Done')

In [ ]:
# Compare unnormalized matrix with normalized
# Mean is subtracted, but this is just to check fp math
#
# Result: don't need to normalize

for normalize in [False, True]:
    X_small = X[:, :100].copy()
    if normalize:
        X_small /= 255

    U, s, Vh = np.linalg.svd(X_small, full_matrices=False)
    X_recon = U.dot(np.diag(s)).dot(Vh)
    if normalize:
        X_recon *= 255
    
    norm = np.mean(np.linalg.norm(X_recon - X[:, :100], None, 0))

    print(('Normalized' if normalize else 'Unnormalized') + f': {norm}')

In [ ]:
# Benchmark different SVD methods
# Roughly sorted by expected duration, descending

k = 100
X_small = X[:, :2*k].copy()

trials = {
    'Truncated SVD  (numpy)    ': lambda: np.linalg.svd(X_small, full_matrices=False),
    'Sparse SVD     (scipy)    ': lambda: scipy.sparse.linalg.svds(X_small, k),
    'Randomized SVD (sklearn)  ': lambda: randomized_svd(X_small, k, random_state=0),
}

for name, fn in trials.items():
    start = time.perf_counter()
    fn()
    print(f'{name}: {time.perf_counter() - start}')

In [ ]:
# Run SVD on whole dataset

k = 800
U, s, Vh = randomized_svd(X, k, random_state=0)

In [ ]:
# Visualize reconstruction of first image
#
# Result: 400 is very good

fig = plt.figure()
rows = 2
cols = 3
grid = ImageGrid(fig, 111, (rows, cols))

ks = [300, 320, 350, 361, 400]

for r in range(rows):
    for c in range(cols):
        if r == 0 and c == 0:
            grid[r * cols + c].imshow((X_mean + X[:, 0]).reshape(h, w), **IMSHOW_PARAMS)
        else:
            i = ks[r * cols + c - 1]
            Xr = U[:, :i].dot(np.diag(s[:i])).dot(Vh[:i, 0])
            grid[r * cols + c].imshow((X_mean + Xr).reshape(h, w), **IMSHOW_PARAMS) 

In [ ]:
# Visualize multiple images at same k
#
# Result: 400 will be fine, although uncertain for faces with rare features

fig = plt.figure()
rows = 5
cols = 5
grid = ImageGrid(fig, 111, (rows, cols))
num_k = 400

for r in range(rows):
    for c in range(cols):
        i = r * cols + c
        Xr = U[:, :num_k].dot(np.diag(s[:num_k])).dot(Vh[:num_k, i])
        grid[i].imshow((X_mean + Xr).reshape(h, w), **IMSHOW_PARAMS) 

In [ ]:
# Create reconstructions of an image using very few modes

fig = plt.figure()
rows = 1
cols = 5
grid = ImageGrid(fig, 111, (rows, cols))

for i in range(cols):
    grid[i].imshow((U[:, :i].dot(np.diag(s[:i])).dot(Vh[:i, 0]) + X_mean).reshape(h, w), **IMSHOW_PARAMS)

In [ ]:
# Convert eigenfaces to RGB
# Red is positive and blue negative (should match mpl's bwr)

num_k = 400
eigenfaces_rgb = np.zeros((h, w, 3, num_k), dtype=np.uint8)
scales = np.zeros(k)

for i in range(num_k):
    eigenface = U[:, i].reshape(h, w).copy()

    # Normalize to [-1, 1]
    scales[i] = np.max(np.abs(eigenface))
    eigenface /= scales[i]

    eigenfaces_rgb[..., 0, i] = 255 - eigenface.clip(max=0) * -255
    eigenfaces_rgb[..., 1, i] = 255 - np.abs(eigenface) * 255
    eigenfaces_rgb[..., 2, i] = 255 - eigenface.clip(min=0) * 255

plt.imshow(eigenfaces_rgb[..., 4], **IMSHOW_PARAMS)

In [ ]:
# Save images and metadata for NFT

print('Saving images...')

attributes = {}

for i in range(num_k):
    attributes[i] = {
        'scale_factor': float(scales[i]),
        'singular_value': float(s[i])
    }

    im = Image.fromarray(eigenfaces_rgb[..., i])
    im.save(f'{OUTPUT_DIR}nft/eigenapes/{i}.png')

print('Done. Saving attribute metadata...')

with open(f'{DATA_DIR}eigenape_attributes.json', 'w') as f:
    json.dump(attributes, f)

print('Metadata saved.')

In [ ]:
# Save images for blog, OpenSea

def make_grid(rows, cols):
    grid = np.zeros((rows * h, cols * w, 3), dtype=np.uint8)

    for r in range(rows):
        for c in range(cols): 
            i = r * cols + c
            grid[r*h:(r+1)*h, c*w:(c+1)*w, :] = eigenfaces_rgb[..., i] 
    
    return grid

eigenfaces_main = make_grid(2, 3)
eigenfaces_banner = make_grid(4, 14)
eigenfaces_all = make_grid(20, 20)[::4, ::4, ...]

f, ax = plt.subplots(3, 1)
ax[0].imshow(eigenfaces_main)
ax[1].imshow(eigenfaces_banner)
ax[2].imshow(eigenfaces_all)

# For blog
im = Image.fromarray(eigenfaces_main)
im.save(f'{OUTPUT_DIR}blog/main_eigenfaces.png')

im = Image.fromarray(eigenfaces_all)
im.save(f'{OUTPUT_DIR}blog/all_eigenfaces.jpg')

# For OpenSea
im = Image.fromarray(eigenfaces_main)
im = im.resize((600, 400))
im.save(f'{OUTPUT_DIR}nft/opensea/highlight.png')

im = Image.fromarray(eigenfaces_banner)
im.resize((1400, 400))
im.save(f'{OUTPUT_DIR}nft/opensea/banner.png')

In [ ]:
# Singular value graphs for blog

energy = np.cumsum(s) / np.sum(s)
e90 = np.searchsorted(energy, .9)
e95 = np.searchsorted(energy, .95)
e99 = np.searchsorted(energy, .99)

f, ax = plt.subplots(1, 2)

ax[0].set_title('Singular Values')
ax[0].set_xlabel('$i$')
ax[0].set_ylabel('$\sigma_i$')
ax[0].set_yscale('log')
ax[0].plot(s)

ax[1].set_title('Cumulative Energy')
ax[1].set_xlabel('$i$')
ax[1].set_ylabel('$\sum_{j=1}^{i} \sigma_j / \sum_{j=1}^{' + f'{k}' + '} \sigma_j$')
ax[1].plot(energy)
ax[1].axvline(e90, c='r')
ax[1].text(e90 + 10, 0.1, '90%', rotation=90)
ax[1].axvline(e95, c='g')
ax[1].text(e95 + 10, 0.1, '95%', rotation=90)
ax[1].axvline(e99, c='b')
ax[1].text(e99 + 10, 0.1, '99%', rotation=90)

f.tight_layout()
f.savefig(f'{OUTPUT_DIR}blog/sing_values.png')

print(f'Cumulative energy at k: {energy[num_k]}')

In [ ]:
# Create graph showing error v k, up to 1000. For blog.
# Warning: slow and memory intensive with high num_tests

num_tests = 25
idxs = np.random.randint(0, X.shape[1], num_tests)
X_idxs = X[:, idxs]
Vh_idxs = Vh[:, idxs]
recons = np.zeros((X.shape[0], num_tests))
norms = np.ndarray((num_tests, k))

for i in range(k):
    recons += np.outer(U[:, i], Vh_idxs[i, :]) * s[i]
    norms[:, i] = np.linalg.norm(recons - X_idxs, None, 0)

f, ax = plt.subplots(1, 1)

ax.plot(norms.T)
ax.set_title('Reconstruction Error vs $i$ Greatest $\sigma_i$, ' + f'{num_tests} Columns')
ax.set_xlabel('$i$')
ax.set_ylabel('$||X-X_i||_2$')
f.savefig(f'{OUTPUT_DIR}blog/norms.png')